In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import numpy as np
import sys
import io

from matplotlib import pyplot as plt
from skimage import color
from skimage.io import imread
from skimage.io import imshow
from scipy.sparse import csr_matrix

import torch
from torch import optim
from torch.nn import functional as tf
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from utils import VOC_LABEL2COLOR
from utils import VOC_STATISTICS
from utils import numpy2torch
from utils import torch2numpy

In [2]:
# VOC Dataset class, derived from PyTorch Dataset
class VOC2007Dataset(Dataset):
    def __init__(self, root, train, num_examples):
        super().__init__()
        
        # Reading num_examples lines from the split file for the image names
        if train:
            split_file = open(root + "/ImageSets/Segmentation/train.txt", "r")
        else:
            split_file = open(root + "/ImageSets/Segmentation/val.txt", "r")
        splits = []
        
        # TODO: Read random line?
        for _ in range(0, num_examples):
            line = split_file.readline()
            splits.append(line[:6])
        split_file.close()
                
        # Reading pictures and gt for the entries in splits
        # Saving a dictionary of {im, gt} for every entry in splits
        self.data = []
        for name in splits:
            self.data.append({
                "im" : numpy2torch(imread(root + "/JPEGImages/{}.jpg".format(name))),
                # TODO: Convert colors in gt to labels ?
                "gt" : numpy2torch(imread(root + "/SegmentationClass/{}.png".format(name)))
            })
        
    def __getitem__(self, index):
        # Returning random dictionary of {im, gt} from dataset
        example_dict = np.random.choice(self.data)
        assert (isinstance(example_dict, dict))
        assert ('im' in example_dict.keys())
        assert ('gt' in example_dict.keys())
        return example_dict

    def __len__(self):
        return len(self.data)

In [3]:
# Create and return pytorch.utils DataLoader
def create_loader(dataset, batch_size, shuffle, num_workers):
    loader = DataLoader(dataset, batch_size, shuffle, num_workers=num_workers)
    assert (isinstance(loader, DataLoader))
    return loader

In [4]:
def voc_label2color(np_image, np_label):
    assert (isinstance(np_image, np.ndarray))
    assert (isinstance(np_label, np.ndarray))

    colored = np.zeros(shape=np_image.shape, dtype=np_image.dtype)
    
    # Iterate over height
    for a in range(0, np_image.shape[0]):
        # Iterate over width
        for b in range(0, np_image.shape[1]):
            # Convert to greyscale PLUS the LABEL2COLOR values
            """
            colored[0, a, b] = 0.3 * np_image[0, a, b] #+ VOC_LABEL2COLOR[np_label[a, b]][0]
            colored[1, a, b] = 0.3 * np_image[1, a, b] #+ VOC_LABEL2COLOR[np_label[a, b]][1]
            colored[2, a, b] = 0.3 * np_image[2, a, b] #+ VOC_LABEL2COLOR[np_label[a, b]][2]
            """
            colored[a, b, 0] = 0.3 * np_image[a, b, 0] + np_label[a, b, 0]
            colored[a, b, 1] = 0.59 * np_image[a, b, 1] + np_label[a, b, 1]
            colored[a, b, 2] = 0.11 * np_image[a, b, 2] + np_label[a, b, 2]

    assert (np.equal(colored.shape, np_image.shape).all())
    assert (np_image.dtype == colored.dtype)
    return colored

In [5]:
def show_dataset_examples(loader, grid_height, grid_width, title):
    plt.figure(figsize=(10, 10))
    # TODO: Fix title and plots
    plt.title(title)

    c = 1
    # Call the data loader until grid_height*grid_width samples are generated
    for dic in loader:
        if c > grid_height*grid_width:
            break
        else:
            plt.subplot(grid_height, grid_width, c)
            plt.plot(np.arange(0, 10), [x**2 for x in np.arange(0, 10)])
            # TODO: Plot result from voc_label2color
            colored_img = voc_label2color(torch2numpy(dic["im"][0,:,:,:]), torch2numpy(dic["gt"][0,:,:,:]))
            plt.imshow(colored_img)
            c += 1
    plt.show()
    pass

In [6]:
def standardize_input(input_tensor):
    
    # Normalize input by (x - mean(x)) / std(x) from VOC_STATISTICS
    normalized = torch.zeros(size=input_tensor.shape)
    
    """
    normalized[0,0,:,:] = (input_tensor[0,0,:,:] - VOC_STATISTICS["mean"][0]) / VOC_STATISTICS["std"][0]
    normalized[0,1,:,:] = (input_tensor[0,1,:,:] - VOC_STATISTICS["mean"][1]) / VOC_STATISTICS["std"][1]
    normalized[0,2,:,:] = (input_tensor[0,2,:,:] - VOC_STATISTICS["mean"][2]) / VOC_STATISTICS["std"][2]
    """
    transformation = transforms.Normalize(
        mean = VOC_STATISTICS["mean"],
        std = VOC_STATISTICS["std"]
    )
    
    normalized[0,:,:,:] = transformation(input_tensor[0,:,:,:].float())
    
    assert (type(input_tensor) == type(normalized))
    assert (input_tensor.size() == normalized.size())
    return normalized

In [7]:
def run_forward_pass(normalized, model):
    
    prediction = []
    acts = []

    assert (isinstance(prediction, torch.Tensor))
    assert (isinstance(acts, torch.Tensor))
    return prediction, acts

In [8]:
def average_precision(prediction, gt):
    return None

In [9]:
def show_inference_examples(loader, model, grid_height, grid_width, title):
    model.eval()
    c = 1
    for dic in loader:
        if c > grid_height*grid_width:
            break
        # Normalize current image
        std_im = standardize_input(dic["im"])
        print("std_im: {} {}".format(std_im.dtype, std_im.shape))
        print(type(model(std_im)))
        c += 1
    pass

In [10]:
def find_unique_example(loader, unique_foreground_label):
    example = []

    assert (isinstance(example, dict))
    return example

In [11]:
def show_unique_example(example_dict, model):
    pass

In [12]:
def show_attack(example_dict, model, src_label, target_label, learning_rate, iterations):
    pass

In [13]:
def problem2():
    # Please set an environment variables 'VOC2007_HOME' pointing to your '../VOCdevkit/VOC2007' folder
    os.environ["VOC2007_HOME"] = "/home/yannik/Computer-Vison-2/asgn4/VOCdevkit/VOC2007"
    root = os.environ["VOC2007_HOME"]

    
    # create datasets for training and validation
    #train_dataset = VOC2007Dataset(root, train=True, num_examples=128)
    train_dataset = VOC2007Dataset(root, train=True, num_examples=32)
    #valid_dataset = VOC2007Dataset(root, train=False, num_examples=128)
    valid_dataset = VOC2007Dataset(root, train=False, num_examples=32)
    
    # create data loaders for training and validation
    # you can safely assume batch_size=1 in our tests..
    train_loader = create_loader(train_dataset, batch_size=1, shuffle=True, num_workers=1)
    valid_loader = create_loader(valid_dataset, batch_size=1, shuffle=False, num_workers=1)

    """
    # show some images for the training and validation set
    show_dataset_examples(train_loader, grid_height=2, grid_width=3, title='training examples')
    show_dataset_examples(valid_loader, grid_height=2, grid_width=3, title='validation examples')

    """
    # Load Deeplab network
    model = models.segmentation.deeplabv3_resnet101(pretrained=True, num_classes=21)

    # Apply deeplab. Switch to training loader if you want more variety.
    show_inference_examples(valid_loader, model, grid_height=2, grid_width=3, title='inference examples')

    """
    # attack1: convert cat to dog
    cat_example = find_unique_example(valid_loader, unique_foreground_label=8)
    show_unique_example(cat_example, model=model)
    show_attack(cat_example, model, src_label=8, target_label=12, learning_rate=1.0, iterations=10)
    """
    
    # feel free to try other examples..

In [14]:
if __name__ == '__main__':
    problem2()

std_im: torch.float32 torch.Size([1, 3, 375, 500])
<class 'collections.OrderedDict'>
std_im: torch.float32 torch.Size([1, 3, 500, 375])
<class 'collections.OrderedDict'>
std_im: torch.float32 torch.Size([1, 3, 375, 500])
<class 'collections.OrderedDict'>
std_im: torch.float32 torch.Size([1, 3, 500, 375])
<class 'collections.OrderedDict'>
std_im: torch.float32 torch.Size([1, 3, 375, 500])
<class 'collections.OrderedDict'>
std_im: torch.float32 torch.Size([1, 3, 375, 500])
<class 'collections.OrderedDict'>
